# 2026-02-14.ipynb

- Reference: https://aubreymoore.github.io/jb/create-one-to-many-sql/

## TODO

- [x] move `get_data_for_images_table` function to `roadside.py`
- [x] move `get_data_for_detections_table` function to `roadside.py`
- [x] remove detection attribute fields from schema
- [x] prevent duplication of records in database

In [1]:
import roadside as rs
import sqlite3
import os
from icecream import ic
import pandas as pd

# Global variables for this run

In [2]:
root_dir = "/home/aubrey/Desktop/sam3-2026-01-31"
image_paths = ["20251129_152106.jpg", "08hs-palms-03-zglw-superJumbo.webp" ]
text_prompts=["coconut palm tree"]

# SQLite3 database
delete_database_before_run = True # normally False; change to True to rebuild db during testing
db_path = 'sam3_detections.sqlite3'
schema_sql = """
--- 2026-02-14 07:32

CREATE TABLE IF NOT EXISTS images (
  image_id INTEGER PRIMARY KEY,
  image_path TEXT UNIQUE,
  image_width INTEGER,
  image_height INTEGER,
  timestamp TEXT,
  latitude REAL,
  longitude REAL
);

CREATE TABLE IF NOT EXISTS detections (
  detection_id INTEGER PRIMARY KEY,
  image_id INTEGER,
  class_id INTEGER,
  poly_wkt TEXT,
  x_min INTEGER,
  y_min INTEGER,
  x_max INTEGER,
  y_max INTEGER,
  confidence REAL,
  FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE 
);
"""

# is_accepted INTEGER NOT NULL DEFAULT 0,
# is_healthy INTEGER NOT NULL DEFAULT 0,
# is_damaged INTEGER NOT NULL DEFAULT 0,
# has_vcuts INTEGER NOT NULL DEFAULT 0,
# is_dead INTEGER NOT NULL DEFAULT 0,
# is_crowded INTEGER NOT NULL DEFAULT 0,
# is_occluded INTEGER NOT NULL DEFAULT 0,
# has_other_problem INTEGER NOT NULL DEFAULT 0,



# Functions

In [3]:
import gc
import torch

def delete_results_from_gpu_memory():
    """
    Explicitly manages memory after processing each image to prevent running out of GPU memory
    """
    global results_gpu
    del results_gpu 
    gc.collect() 
    torch.cuda.empty_cache() # Clears unoccupied cached memory

# Usage example:
    
# delete_results_from_gpu_memory()


# Main

In [4]:
if delete_database_before_run and os.path.exists(db_path):
    os.remove(db_path)

In [5]:
assert rs.check_gpu(), 'ERROR: GPU is unavailable.'

CUDA version: 12.8
GPU device name: NVIDIA GeForce RTX 3080 Laptop GPU


In [6]:
# connect to db. A new db is created if db_path does not exist.
con = sqlite3.connect(db_path)

# create new db if one does not exist; otherwise tables are not modified
con.executescript(schema_sql);

In [7]:
for image_path in image_paths:
    
    # skip image if it is already in the database
    if con.execute(f'SELECT COUNT(*) FROM images WHERE  image_path = "{image_path}"').fetchone()[0] > 0:
        print(f'WARNING: Image {image_path} is already in the database. Skipping to next image.')
        continue
        
    # Detect objects in image
    results_gpu = rs.run_sam3_semantic_predictor(input_image_path=image_path, text_prompts=text_prompts)
    
    # Free up GPU memory in preparation for detecting objects in the next image
    # This is a work-around to prevent out-of-memory errors from the GPU
    # I move all results for further processing and use the GPU only for object detection.
    results_cpu = [r.cpu() for r in results_gpu] # copy results to CPU
    print('deleting results from GPU memory')       
    delete_results_from_gpu_memory() # Clear GPU memory after processing each image
    
    # populate 'images' table
    image_width, image_height, timestamp, latitude, longitude = rs.get_data_for_images_table(results_cpu, image_path)
    sql = """
    INSERT INTO images (image_path, image_width, image_height, timestamp, latitude, longitude) 
    VALUES (?,?,?,?,?,?)
    RETURNING image_id
    """
    parameters = (image_path, image_width, image_height, timestamp, latitude, longitude,)
    ic(parameters)
    
    # sql = 'INSERT INTO images (image_path) VALUES (?) RETURNING image_id'
    try:
        image_id = con.execute(sql, parameters).fetchone()[0] # THE COMMA IN THE PARAMETERS TUPLE IS IMPORTANT
    except sqlite3.IntegrityError as e:
        print(f'ERROR: Image {image_path} already exists in {db_path}')
        raise e    
    con.commit()
    ic(image_id)
    
    # populate 'detections' table
    # df_detections = rs.get_data_for_detections_table(results_cpu, image_id)
    df_detections = rs.get_data_for_detections_table(results_cpu, image_id)
    for i, r in df_detections.iterrows():
            # populate 'detections' table
        sql = ''' 
        INSERT INTO detections
            (image_id, class_id, poly_wkt, x_min, y_min, x_max, y_max, confidence)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?);
        '''
        parameters = (image_id, 0, r['poly_wkt'], r['x_min'], r['y_min'], r['x_max'], r['y_max'], r['confidence']) 
        con.execute(sql, parameters)
        con.commit()
con.close()   
print('FINISHED')    

Ultralytics 8.4.9 🚀 Python-3.13.11 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 15992MiB)

image 1/1 /home/aubrey/Desktop/sam3-2026-01-31/20251129_152106.jpg: 1932x1932 25 coconut palm trees, 1230.5ms
Speed: 14.8ms preprocess, 1230.5ms inference, 5.5ms postprocess per image at shape (1, 3, 1932, 1932)
Results saved to /home/aubrey/Desktop/blog2026/runs/segment/predict172
deleting results from GPU memory


ic| parameters: ('20251129_152106.jpg',
                 1920,
                 1080,
                 '2025:11:29 15:21:06',
                 -17.73039628333333,
                 168.18940991666668)
ic| image_id: 1


Ultralytics 8.4.9 🚀 Python-3.13.11 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 15992MiB)

image 1/1 /home/aubrey/Desktop/sam3-2026-01-31/08hs-palms-03-zglw-superJumbo.webp: 1932x1932 2 coconut palm trees, 1129.7ms
Speed: 13.6ms preprocess, 1129.7ms inference, 0.9ms postprocess per image at shape (1, 3, 1932, 1932)
Results saved to /home/aubrey/Desktop/blog2026/runs/segment/predict173
deleting results from GPU memory
FINISHED


ic| parameters: ('08hs-palms-03-zglw-superJumbo.webp', 1366, 2048, None, None, None)
ic| image_id: 2


# Retrieve data from database

In [8]:
con = sqlite3.connect(db_path)
pd.read_sql('SELECT * FROM images', con)

,image_id,image_path,image_width,image_height,timestamp,latitude,longitude
0,1,20251129_152106.jpg,1920,1080,2025:11:29 15:21:06,-17.730396,168.18941
1,2,08hs-palms-03-zglw-superJumbo.webp,1366,2048,NaN,NaN,NaN


In [9]:
pd.read_sql('SELECT * FROM detections', con)

,detection_id,image_id,class_id,poly_wkt,x_min,y_min,x_max,y_max,confidence
0,1,1,0,"POLYGON ((1339 805, 1338 804, 1338 803, 1337 8...",1342,709,1462,992,0.649902
1,2,1,0,"POLYGON ((437 934, 437 935, 438 936, 439 936, ...",266,762,338,944,0.259521
2,3,1,0,"POLYGON ((538 795, 537 796, 535 796, 534 797, ...",491,794,561,961,0.698730
3,4,1,0,"POLYGON ((902 863, 902 865, 901 866, 901 867, ...",817,790,938,971,0.743164
4,5,1,0,"POLYGON ((906 838, 906 835, 906 838, 943 847, ...",938,801,963,990,0.479980
5,6,1,0,"POLYGON ((1681 715, 1680 716, 1680 717, 1675 7...",1592,715,1778,905,0.771484
6,7,1,0,"POLYGON ((553 936, 553 938, 554 939, 554 940, ...",557,801,682,958,0.348389
7,8,1,0,"POLYGON ((1130 942, 1130 945, 1129 946, 1129 9...",1046,801,1136,977,0.677734
8,9,1,0,"POLYGON ((1519 750, 1519 751, 1518 752, 1518 7...",1483,748,1624,902,0.616211
9,10,1,0,"POLYGON ((1013 829, 1008 834, 1006 834, 1003 8...",963,828,1055,985,0.763672


In [10]:
con.close()